## Andriod Mobile Grade Prediction


## Fetching and Data Cleaning

**Importing the Nessary libraries**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

**Reading the Data from the FullData.xlsx file**
*This file consists of all the important coloums needed for the prediction*
[Click here to get the file](https://docs.google.com/spreadsheets/d/1PkmDSV-E9dex5pFUO7tlazElxOByDrxS/edit?usp=sharing&ouid=115307969613615218667&rtpof=true&sd=true)

In [2]:
andriod_data = pd.read_excel('/content/fulldata.xlsx')

**Seeing the contents of the data**

In [3]:
andriod_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15467 entries, 0 to 15466
Data columns (total 32 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Grade                                                15467 non-null  object 
 1   IMEI                                                 15467 non-null  int64  
 2   Erased                                               15467 non-null  bool   
 3   Passed                                               15464 non-null  object 
 4   Failed                                               2159 non-null   object 
 5   TestPlan                                             15467 non-null  object 
 6   cosmetic_grade                                       14071 non-null  object 
 7   b_and_p_qual                                         14071 non-null  float64
 8   blocklist                                            14006 non-nul

In [6]:
andriod_data.head()

,Grade,IMEI,Erased,Passed,Failed,TestPlan,cosmetic_grade,b_and_p_qual,blocklist,ardroid device_info.IMEI,...,ardroid device_info.battery_health_percentage,ardroid device_info.battery_total_operating_time,ardroid device_info.original_display,ardroid device_info.original_battery,ardroid device_info.camera,ardroid device_info.battery,ardroid device_info.display,ardroid device_info.activation_fmip_lock,ardroid device_info.sys_log_battery_health,ardroid device_info.battery_health_metric
0,C+,353165808227304,True,"VIBRATOR,LOUD_SPEAKER,EAR_SPEAKER,ALL_MICS,HEA...",NaN,wholesale,C,0.0,0.0,353165808227304,...,86,20347,yes,yes,Genuine,Genuine,Genuine,False,86.0,212
1,C+,354073559622690,True,"VIBRATOR,LOUD_SPEAKER,EAR_SPEAKER,ALL_MICS,HEA...",NaN,wholesale,C,0.0,0.0,354073559622690,...,85,21742,yes,yes,Genuine,Genuine,Genuine,False,85.0,209
2,C+,351228765631934,True,"VIBRATOR,LOUD_SPEAKER,EAR_SPEAKER,ALL_MICS,HEA...",NaN,wholesale,C,0.0,0.0,351228765631934,...,100,21455,yes,yes,Genuine,Genuine,Genuine,False,101.0,129
3,C+,351228761821877,True,"VIBRATOR,LOUD_SPEAKER,EAR_SPEAKER,ALL_MICS,HEA...",NaN,wholesale,C,0.0,0.0,351228761821877,...,86,22514,yes,yes,Genuine,Genuine,Genuine,False,86.0,175
4,C+,351228760682718,True,"VIBRATOR,LOUD_SPEAKER,EAR_SPEAKER,ALL_MICS,HEA...",NaN,wholesale,C,0.0,0.0,351228760682718,...,86,22001,yes,yes,Genuine,Genuine,Genuine,False,86.0,334


**Filtering out only the required colums those are inportand and needed**

In [7]:
andriod_data = andriod_data[['Grade',
                             'Passed',
                             'ardroid device_info.battery_cycle_count' ,
                             'ardroid device_info.battery_temperature' ,
                             'ardroid device_info.battery_health_percentage' ,
                             'ardroid device_info.battery_total_operating_time']]

**Renameing the colums**

In [8]:
andriod_data.rename(columns={'ardroid device_info.battery_temperature': 'battery_temperature' ,
                            'ardroid device_info.battery_health_percentage' :'battery_health_percentage' ,
                            'ardroid device_info.battery_total_operating_time' :'battery_total_operating_time',
                            'ardroid device_info.battery_cycle_count':'battery_cycle_count'}, inplace=True)

**Visualization of the Data**

In [9]:
andriod_data.head()

,Grade,Passed,battery_cycle_count,battery_temperature,battery_health_percentage,battery_total_operating_time
0,C+,"VIBRATOR,LOUD_SPEAKER,EAR_SPEAKER,ALL_MICS,HEA...",639,2619,86,20347
1,C+,"VIBRATOR,LOUD_SPEAKER,EAR_SPEAKER,ALL_MICS,HEA...",451,2689,85,21742
2,C+,"VIBRATOR,LOUD_SPEAKER,EAR_SPEAKER,ALL_MICS,HEA...",159,2659,100,21455
3,C+,"VIBRATOR,LOUD_SPEAKER,EAR_SPEAKER,ALL_MICS,HEA...",569,2700,86,22514
4,C+,"VIBRATOR,LOUD_SPEAKER,EAR_SPEAKER,ALL_MICS,HEA...",648,2829,86,22001


## Preprocessing

**Encoding the unique values of the Grade in numbers**

In [10]:
lableEncoder = LabelEncoder()
andriod_data['Grade'] = lableEncoder.fit_transform(andriod_data['Grade'])

**Scaleing all neumeric values within same scale**

In [11]:
standardScaler = StandardScaler()
columns_to_scale = ['battery_cycle_count', 'battery_temperature', 'battery_health_percentage', 'battery_total_operating_time'] # Select the columns you want to scale
andriod_data[columns_to_scale] = standardScaler.fit_transform(andriod_data[columns_to_scale])

**Transforming a column with categorical values into multiple columns (each representing a category)**

In [12]:
andriod_data = andriod_data.join(andriod_data['Passed'].str.get_dummies(sep=','))
andriod_data.drop('Passed', axis=1, inplace=True) #Removing the Passed coloum as it is mo longer needed

**Randomly shuffle (reorder) the rows**

In [13]:
andriod_data = andriod_data.sample(frac=1, random_state=42).reset_index(drop=True)

**Visualization**

In [14]:
andriod_data.head()

,Grade,battery_cycle_count,battery_temperature,battery_health_percentage,battery_total_operating_time,ACCELEROMETER,ALL_MICS,BLUETOOTH,DISPLAY,DISPLAY_BURN_IN,...,MULTITOUCH,NFC,PROXIMITY,REAR_CAMERA,TOUCH,TRUE_DEPTH_CAMERA,VIBRATE_SWITCH,VIBRATOR,WIREDCHARGING,WIRELESSCHARGING
0,14,1.772878,0.343681,-1.406351,0.945217,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
1,2,1.880786,-0.481049,-1.406351,1.290096,1,1,1,0,1,...,1,0,1,1,1,1,1,1,1,1
2,14,1.142218,-0.713005,-0.968641,1.007358,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
3,0,-0.557928,0.395227,-0.093219,-0.277457,1,1,1,0,1,...,1,0,1,1,1,1,1,1,1,1
4,4,-0.342112,0.343681,0.198588,-0.653406,1,1,1,0,1,...,1,0,1,1,1,1,1,1,1,1


## Splliting and Removal of imbalance in data

**Split data into features and target**

In [15]:
X = andriod_data.drop('Grade', axis=1)
y = andriod_data['Grade']

**Showing imbalance in data**

In [16]:
y.value_counts()

,count
Grade,
4,4184
14,2265
0,2007
12,1598
8,1580
2,982
10,673
13,519
5,514


**Removing the imbalance in data**

In [17]:
X_resampled, y_resampled = SMOTE().fit_resample(X, y) #SMOTE uses KNN architecture to create new data and removes to imbalance

**Showing rectification of imbalance in data**

In [19]:
y_resampled.value_counts()

,count
Grade,
14,4184
2,4184
0,4184
4,4184
13,4184
8,4184
12,4184
11,4184
15,4184


**Splitting it to test and train**

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

## Model trainning

## **Random forest**

In [21]:
from sklearn.ensemble import RandomForestClassifier

**initialization and trainning**

In [22]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

**Prediction**

In [27]:
rf_classifier_pred = rf_classifier.predict(X_test)

**Accuracy and Classification Report**

In [28]:
accuracy = accuracy_score(y_test, rf_classifier_pred)
print(f"Accuracy: {accuracy}")

classification_rep = classification_report(y_test, rf_classifier_pred)
print("Classification Report:")
print(classification_rep)

Accuracy: 0.8851295839868549
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.78      0.75       807
           1       0.98      0.99      0.98       832
           2       0.78      0.75      0.76       840
           3       0.98      0.99      0.98       858
           4       0.69      0.62      0.66       809
           5       0.97      0.96      0.96       853
           6       0.98      0.99      0.99       856
           7       0.99      1.00      1.00       835
           8       0.84      0.80      0.82       868
           9       0.97      0.95      0.96       856
          10       0.77      0.89      0.83       791
          11       0.96      0.98      0.97       831
          12       0.80      0.82      0.81       827
          13       0.98      0.95      0.96       870
          14       0.79      0.73      0.76       853
          15       0.94      0.95      0.94       803

    accuracy                

## Logistic Regression

In [25]:
from sklearn.linear_model import LogisticRegression

**initialization and trainning**

In [26]:
logistic_regression = LogisticRegression(max_iter=1000)
logistic_regression.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

**Prediction**

In [29]:
logistic_regression_pred = logistic_regression.predict(X_test)

**Accuracy and Classification Report**

In [31]:
accuracy = accuracy_score(y_test, logistic_regression_pred)
print(f"Accuracy: {accuracy}")

classification_rep = classification_report(y_test, logistic_regression_pred)
print("Classification Report:")
print(classification_rep)

Accuracy: 0.505638957353051
Classification Report:
              precision    recall  f1-score   support

           0       0.35      0.60      0.44       807
           1       0.60      0.69      0.64       832
           2       0.30      0.16      0.21       840
           3       0.51      0.42      0.46       858
           4       0.35      0.43      0.39       809
           5       0.60      0.51      0.55       853
           6       0.54      0.48      0.51       856
           7       0.97      1.00      0.98       835
           8       0.63      0.46      0.53       868
           9       0.55      0.32      0.40       856
          10       0.37      0.39      0.38       791
          11       0.51      0.53      0.52       831
          12       0.52      0.56      0.54       827
          13       0.52      0.51      0.51       870
          14       0.36      0.45      0.40       853
          15       0.53      0.60      0.56       803

    accuracy                 

## Using Support Vector Machines (SVM)

In [32]:
from sklearn.svm import SVC

**initialization and trainning**

In [33]:
support_vector_machine = SVC(random_state = 42)
support_vector_machine.fit(X_train, y_train)

SVC(random_state=42)

**Prediction**

In [34]:
support_vector_machine_pred = support_vector_machine.predict(X_test)

**Accuracy and Classification Report**

In [35]:
accuracy = accuracy_score(y_test, support_vector_machine_pred)
print(f"Accuracy: {accuracy}")

classification_rep = classification_report(y_test, support_vector_machine_pred)
print("Classification Report:")
print(classification_rep)

Accuracy: 0.6589737844499216
Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.58      0.54       807
           1       0.80      0.86      0.83       832
           2       0.39      0.23      0.29       840
           3       0.79      0.76      0.78       858
           4       0.41      0.53      0.46       809
           5       0.76      0.74      0.75       853
           6       0.75      0.76      0.76       856
           7       0.99      1.00      0.99       835
           8       0.72      0.45      0.55       868
           9       0.81      0.65      0.72       856
          10       0.41      0.64      0.50       791
          11       0.70      0.87      0.78       831
          12       0.60      0.65      0.62       827
          13       0.84      0.65      0.73       870
          14       0.48      0.42      0.45       853
          15       0.72      0.76      0.74       803

    accuracy                

## K-Nearest Neighbors (KNN)

In [36]:
from sklearn.neighbors import KNeighborsClassifier

**initialization and trainning**

In [37]:
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(X_train, y_train)

KNeighborsClassifier()

**Prediction**

In [38]:
knn_classifier_pred = knn_classifier.predict(X_test)

**Accuracy and Classification Report**

In [39]:
accuracy = accuracy_score(y_test, knn_classifier_pred)
print(f"Accuracy: {accuracy}")

classification_rep = classification_report(y_test, knn_classifier_pred)
print("Classification Report:")
print(classification_rep)

Accuracy: 0.846366420195683
Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.75      0.70       807
           1       0.97      0.98      0.97       832
           2       0.66      0.78      0.72       840
           3       0.94      0.97      0.95       858
           4       0.68      0.40      0.50       809
           5       0.94      0.92      0.93       853
           6       0.92      1.00      0.96       856
           7       0.99      1.00      1.00       835
           8       0.75      0.74      0.74       868
           9       0.95      0.93      0.94       856
          10       0.71      0.91      0.79       791
          11       0.93      0.97      0.95       831
          12       0.77      0.77      0.77       827
          13       0.96      0.93      0.94       870
          14       0.75      0.56      0.64       853
          15       0.93      0.93      0.93       803

    accuracy                 

## XGBoost

In [40]:
from xgboost import XGBClassifier

**initialization and trainning**

In [41]:
xgboost_classifier = XGBClassifier(random_state = 42)
xgboost_classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

**Prediciton**

In [42]:
xgboost_classifier_pred = xgboost_classifier.predict(X_test)

**Accuracy and Classification Report**

In [43]:
accuracy = accuracy_score(y_test, xgboost_classifier_pred)
print(f"Accuracy: {accuracy}")

classification_rep = classification_report(y_test, xgboost_classifier_pred)
print("Classification Report:")
print(classification_rep)

Accuracy: 0.8059601165135559
Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.64      0.64       807
           1       0.95      0.98      0.97       832
           2       0.65      0.53      0.58       840
           3       0.94      0.95      0.95       858
           4       0.60      0.65      0.62       809
           5       0.92      0.90      0.91       853
           6       0.92      0.98      0.95       856
           7       0.99      1.00      1.00       835
           8       0.76      0.57      0.65       868
           9       0.94      0.89      0.92       856
          10       0.54      0.78      0.64       791
          11       0.91      0.97      0.94       831
          12       0.70      0.73      0.72       827
          13       0.94      0.89      0.91       870
          14       0.61      0.51      0.56       853
          15       0.89      0.91      0.90       803

    accuracy                

## Naive Bayes

In [44]:
from sklearn.naive_bayes import GaussianNB

**initialization and trainning**

In [45]:
gaussian_naive_bayes = GaussianNB()
gaussian_naive_bayes.fit(X_train, y_train)

GaussianNB()

**Prediction**

In [46]:
gaussian_naive_bayes_pred = gaussian_naive_bayes.predict(X_test)

**Accuracy and Classification Report**

In [47]:
accuracy = accuracy_score(y_test, gaussian_naive_bayes_pred)
print(f"Accuracy: {accuracy}")

classification_rep = classification_report(y_test, gaussian_naive_bayes_pred)
print("Classification Report:")
print(classification_rep)

Accuracy: 0.2694002539398013
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       807
           1       0.40      0.08      0.14       832
           2       0.14      0.10      0.12       840
           3       0.29      0.01      0.01       858
           4       0.00      0.00      0.00       809
           5       0.46      0.18      0.26       853
           6       0.54      0.50      0.52       856
           7       0.90      0.82      0.85       835
           8       0.15      0.57      0.24       868
           9       0.13      0.00      0.00       856
          10       0.22      0.46      0.30       791
          11       0.18      0.95      0.30       831
          12       0.00      0.00      0.00       827
          13       0.33      0.00      0.00       870
          14       0.39      0.58      0.46       853
          15       0.40      0.05      0.10       803

    accuracy                

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Prediction

In [48]:
test_data = pd.read_excel('/content/fulldata.xlsx')

In [49]:
test_data = test_data[['Grade',
                             'Passed',
                             'ardroid device_info.battery_cycle_count' ,
                             'ardroid device_info.battery_temperature' ,
                             'ardroid device_info.battery_health_percentage' ,
                             'ardroid device_info.battery_total_operating_time']]

In [50]:
test_data.rename(columns={'ardroid device_info.battery_temperature': 'battery_temperature' ,
                            'ardroid device_info.battery_health_percentage' :'battery_health_percentage' ,
                            'ardroid device_info.battery_total_operating_time' :'battery_total_operating_time',
                            'ardroid device_info.battery_cycle_count':'battery_cycle_count'}, inplace=True)

In [51]:
test_data.iloc[[10064]]

,Grade,Passed,battery_cycle_count,battery_temperature,battery_health_percentage,battery_total_operating_time
10064,S4+,"VIBRATOR,LOUD_SPEAKER,EAR_SPEAKER,ALL_MICS,HEA...",1381,2759,77,34821


In [52]:
data_to_predict = test_data.iloc[[10064]]

In [53]:
data_to_predict = data_to_predict.drop('Grade', axis=1)

In [54]:
data_to_predict

,Passed,battery_cycle_count,battery_temperature,battery_health_percentage,battery_total_operating_time
10064,"VIBRATOR,LOUD_SPEAKER,EAR_SPEAKER,ALL_MICS,HEA...",1381,2759,77,34821


In [55]:
data_to_predict = data_to_predict.join(data_to_predict['Passed'].str.get_dummies(sep=','))
data_to_predict.drop('Passed', axis=1, inplace=True)
columns_to_scale = ['battery_cycle_count', 'battery_temperature', 'battery_health_percentage', 'battery_total_operating_time']
data_to_predict[columns_to_scale] = standardScaler.transform(data_to_predict[columns_to_scale]) #use only standardScaler.transform and not standardScaler.fit_transform as the fit was already done before hand

In [56]:
data_to_predict

,battery_cycle_count,battery_temperature,battery_health_percentage,battery_total_operating_time,ACCELEROMETER,ALL_MICS,BLUETOOTH,DISPLAY_BURN_IN,DISPLAY_DARK_PIXEL,DISPLAY_DARK_REGION,...,MULTITOUCH,NFC,PROXIMITY,REAR_CAMERA,TOUCH,TRUE_DEPTH_CAMERA,VIBRATE_SWITCH,VIBRATOR,WIREDCHARGING,WIRELESSCHARGING
10064,1.640991,-0.635686,-1.260448,1.012453,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


**Get the column names that the random forest was trained on**

In [57]:
training_data_columns = rf_classifier.feature_names_in_

**Reindex data_to_predict to match the training data columns and Fill missing columns with 0 - adjust this based on your data**

In [58]:
data_to_predict = data_to_predict.reindex(columns=training_data_columns, fill_value=0)

In [59]:
y_pred_test = rf_classifier.predict(data_to_predict)

In [60]:
y_pred_test

array([14])

**Use to fetch the name of the class**

In [61]:
lableEncoder.inverse_transform(y_pred_test)

array(['S4+'], dtype=object)

In [62]:
lableEncoder.classes_ #Shows all the classes in lable encoder

array(['A+', 'A-', 'B+', 'B-', 'C+', 'C-', 'D+', 'D-', 'S1+', 'S1-',
       'S2+', 'S2-', 'S3+', 'S3-', 'S4+', 'S4-'], dtype=object)